In [1]:
# https://microsoft.github.io/sql-ml-tutorials/python/rentalprediction/step/2.html

In [2]:
import argparse
import pandas
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
#BOT_FILE='csv/bot_100k.csv'
BOT_FILE='csv/bot_2m.csv'
# HUM_FILE='csv/hum_100k.csv'
HUM_FILE='csv/hum_2m.csv'
# HUM_FILE='csv/human.csv'

In [4]:
bot_dataframe = pandas.read_csv(BOT_FILE, header=None, error_bad_lines=False)
hum_dataframe = pandas.read_csv(HUM_FILE, header=None, error_bad_lines=False)

b'Skipping line 35: expected 11 fields, saw 12\nSkipping line 40: expected 11 fields, saw 12\nSkipping line 177: expected 11 fields, saw 12\nSkipping line 298: expected 11 fields, saw 12\nSkipping line 536: expected 11 fields, saw 12\nSkipping line 549: expected 11 fields, saw 12\nSkipping line 665: expected 11 fields, saw 12\nSkipping line 712: expected 11 fields, saw 12\nSkipping line 760: expected 11 fields, saw 12\nSkipping line 1033: expected 11 fields, saw 12\nSkipping line 1107: expected 11 fields, saw 12\nSkipping line 1112: expected 11 fields, saw 12\nSkipping line 1249: expected 11 fields, saw 12\nSkipping line 1370: expected 11 fields, saw 12\nSkipping line 1608: expected 11 fields, saw 12\nSkipping line 1621: expected 11 fields, saw 12\nSkipping line 1737: expected 11 fields, saw 12\nSkipping line 1784: expected 11 fields, saw 12\nSkipping line 1832: expected 11 fields, saw 12\nSkipping line 2105: expected 11 fields, saw 12\nSkipping line 2227: expected 11 fields, saw 12\nS

b'Skipping line 198217: expected 11 fields, saw 12\nSkipping line 198219: expected 11 fields, saw 12\nSkipping line 198220: expected 11 fields, saw 12\nSkipping line 198221: expected 11 fields, saw 12\nSkipping line 198222: expected 11 fields, saw 12\nSkipping line 198223: expected 11 fields, saw 12\nSkipping line 198224: expected 11 fields, saw 12\nSkipping line 198225: expected 11 fields, saw 12\nSkipping line 198227: expected 11 fields, saw 12\nSkipping line 198231: expected 11 fields, saw 12\nSkipping line 198251: expected 11 fields, saw 12\nSkipping line 198268: expected 11 fields, saw 12\nSkipping line 199370: expected 11 fields, saw 12\nSkipping line 199397: expected 11 fields, saw 12\nSkipping line 199413: expected 11 fields, saw 12\nSkipping line 199414: expected 11 fields, saw 12\nSkipping line 199417: expected 11 fields, saw 12\nSkipping line 200411: expected 11 fields, saw 12\nSkipping line 200453: expected 11 fields, saw 12\nSkipping line 200472: expected 11 fields, saw 12

b'Skipping line 396401: expected 11 fields, saw 12\nSkipping line 396443: expected 11 fields, saw 12\nSkipping line 396462: expected 11 fields, saw 12\nSkipping line 397630: expected 11 fields, saw 12\nSkipping line 397641: expected 11 fields, saw 12\nSkipping line 397670: expected 11 fields, saw 12\nSkipping line 397791: expected 11 fields, saw 12\nSkipping line 398034: expected 11 fields, saw 12\nSkipping line 398061: expected 11 fields, saw 12\nSkipping line 398068: expected 11 fields, saw 12\nSkipping line 398076: expected 11 fields, saw 12\nSkipping line 398211: expected 11 fields, saw 12\nSkipping line 398272: expected 11 fields, saw 12\nSkipping line 398380: expected 11 fields, saw 12\nSkipping line 398435: expected 11 fields, saw 12\nSkipping line 398438: expected 11 fields, saw 12\nSkipping line 399422: expected 11 fields, saw 12\nSkipping line 399486: expected 11 fields, saw 12\nSkipping line 399491: expected 11 fields, saw 12\nSkipping line 399531: expected 11 fields, saw 12

b'Skipping line 595412: expected 11 fields, saw 12\nSkipping line 595476: expected 11 fields, saw 12\nSkipping line 595481: expected 11 fields, saw 12\nSkipping line 595521: expected 11 fields, saw 12\nSkipping line 595526: expected 11 fields, saw 12\nSkipping line 595663: expected 11 fields, saw 12\nSkipping line 595784: expected 11 fields, saw 12\nSkipping line 596022: expected 11 fields, saw 12\nSkipping line 596035: expected 11 fields, saw 12\nSkipping line 596151: expected 11 fields, saw 12\nSkipping line 596198: expected 11 fields, saw 12\nSkipping line 596246: expected 11 fields, saw 12\nSkipping line 596519: expected 11 fields, saw 12\nSkipping line 596566: expected 11 fields, saw 12\nSkipping line 596644: expected 11 fields, saw 12\nSkipping line 596645: expected 11 fields, saw 12\nSkipping line 596694: expected 11 fields, saw 12\nSkipping line 596708: expected 11 fields, saw 12\nSkipping line 596729: expected 11 fields, saw 12\nSkipping line 596747: expected 11 fields, saw 12

b'Skipping line 792795: expected 11 fields, saw 12\nSkipping line 792837: expected 11 fields, saw 12\nSkipping line 792852: expected 11 fields, saw 12\nSkipping line 792861: expected 11 fields, saw 12\nSkipping line 793303: expected 11 fields, saw 12\nSkipping line 793306: expected 11 fields, saw 12\nSkipping line 793308: expected 11 fields, saw 12\nSkipping line 793311: expected 11 fields, saw 12\nSkipping line 793333: expected 11 fields, saw 12\nSkipping line 793358: expected 11 fields, saw 12\nSkipping line 793436: expected 11 fields, saw 12\nSkipping line 793437: expected 11 fields, saw 12\nSkipping line 793486: expected 11 fields, saw 12\nSkipping line 793500: expected 11 fields, saw 12\nSkipping line 793521: expected 11 fields, saw 12\nSkipping line 793539: expected 11 fields, saw 12\nSkipping line 793573: expected 11 fields, saw 12\nSkipping line 793597: expected 11 fields, saw 12\nSkipping line 793639: expected 11 fields, saw 12\nSkipping line 793654: expected 11 fields, saw 12

...

b'Skipping line 393674: expected 11 fields, saw 12\nSkipping line 402152: expected 11 fields, saw 13\nSkipping line 408662: expected 11 fields, saw 13\nSkipping line 411943: expected 11 fields, saw 13\nSkipping line 412343: expected 11 fields, saw 13\nSkipping line 417172: expected 11 fields, saw 12\nSkipping line 419212: expected 11 fields, saw 12\nSkipping line 420807: expected 11 fields, saw 12\nSkipping line 423200: expected 11 fields, saw 13\nSkipping line 425073: expected 11 fields, saw 12\nSkipping line 429719: expected 11 fields, saw 12\nSkipping line 429966: expected 11 fields, saw 12\nSkipping line 430220: expected 11 fields, saw 12\nSkipping line 431810: expected 11 fields, saw 12\nSkipping line 438526: expected 11 fields, saw 12\nSkipping line 439145: expected 11 fields, saw 13\nSkipping line 442379: expected 11 fields, saw 12\nSkipping line 443436: expected 11 fields, saw 12\nSkipping line 443624: expected 11 fields, saw 12\nSkipping line 445513: expected 11 fields, saw 12

b'Skipping line 787058: expected 11 fields, saw 13\nSkipping line 789295: expected 11 fields, saw 13\nSkipping line 789716: expected 11 fields, saw 13\nSkipping line 794828: expected 11 fields, saw 13\nSkipping line 798284: expected 11 fields, saw 13\nSkipping line 798936: expected 11 fields, saw 14\nSkipping line 800568: expected 11 fields, saw 12\nSkipping line 801813: expected 11 fields, saw 12\nSkipping line 806498: expected 11 fields, saw 12\nSkipping line 807353: expected 11 fields, saw 13\nSkipping line 813987: expected 11 fields, saw 13\nSkipping line 815361: expected 11 fields, saw 13\nSkipping line 820422: expected 11 fields, saw 13\nSkipping line 822771: expected 11 fields, saw 13\nSkipping line 823328: expected 11 fields, saw 12\nSkipping line 823439: expected 11 fields, saw 12\nSkipping line 824114: expected 11 fields, saw 13\nSkipping line 824566: expected 11 fields, saw 12\nSkipping line 824687: expected 11 fields, saw 13\nSkipping line 825915: expected 11 fields, saw 12

b'Skipping line 1183858: expected 11 fields, saw 13\nSkipping line 1184256: expected 11 fields, saw 12\nSkipping line 1186007: expected 11 fields, saw 12\nSkipping line 1187038: expected 11 fields, saw 12\nSkipping line 1191720: expected 11 fields, saw 12\nSkipping line 1192804: expected 11 fields, saw 13\nSkipping line 1195662: expected 11 fields, saw 13\nSkipping line 1196628: expected 11 fields, saw 13\nSkipping line 1200167: expected 11 fields, saw 13\nSkipping line 1202432: expected 11 fields, saw 13\nSkipping line 1205069: expected 11 fields, saw 12\nSkipping line 1205547: expected 11 fields, saw 13\nSkipping line 1205665: expected 11 fields, saw 13\nSkipping line 1217297: expected 11 fields, saw 12\nSkipping line 1219616: expected 11 fields, saw 13\nSkipping line 1219800: expected 11 fields, saw 13\nSkipping line 1219934: expected 11 fields, saw 12\nSkipping line 1225402: expected 11 fields, saw 12\nSkipping line 1229111: expected 11 fields, saw 13\nSkipping line 1230383: expect

b'Skipping line 1576966: expected 11 fields, saw 13\nSkipping line 1577324: expected 11 fields, saw 13\nSkipping line 1577627: expected 11 fields, saw 13\nSkipping line 1578871: expected 11 fields, saw 12\nSkipping line 1581665: expected 11 fields, saw 13\nSkipping line 1583280: expected 11 fields, saw 13\nSkipping line 1584206: expected 11 fields, saw 13\nSkipping line 1584885: expected 11 fields, saw 12\nSkipping line 1585783: expected 11 fields, saw 13\nSkipping line 1586537: expected 11 fields, saw 13\nSkipping line 1587357: expected 11 fields, saw 13\nSkipping line 1588618: expected 11 fields, saw 12\nSkipping line 1589765: expected 11 fields, saw 13\nSkipping line 1590024: expected 11 fields, saw 12\nSkipping line 1590129: expected 11 fields, saw 12\nSkipping line 1591656: expected 11 fields, saw 13\nSkipping line 1592921: expected 11 fields, saw 12\nSkipping line 1594106: expected 11 fields, saw 12\nSkipping line 1596138: expected 11 fields, saw 13\nSkipping line 1598295: expect

b'Skipping line 1967957: expected 11 fields, saw 12\nSkipping line 1972079: expected 11 fields, saw 12\nSkipping line 1973034: expected 11 fields, saw 13\nSkipping line 1973926: expected 11 fields, saw 13\nSkipping line 1973931: expected 11 fields, saw 12\nSkipping line 1975176: expected 11 fields, saw 12\nSkipping line 1975994: expected 11 fields, saw 12\nSkipping line 1976473: expected 11 fields, saw 13\n'
/Users/rain/git/bot-data/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
bot_dataframe['target'] = 1
hum_dataframe['target'] = 0

In [6]:
dataset = pandas.concat([bot_dataframe, hum_dataframe]).sample(frac=1).reset_index(drop=True)
dataset

,0,1,2,3,4,5,6,7,8,9,10,target
0,39.64.155.235,NaN,NaN,2018-02-11 04:16:19,GET,/apache-log/access.log,HTTP/1.1,206.0,40960,http://www.almhuette-raith.at/apache-log,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,0
1,198.50.156.189,NaN,NaN,2017-09-10 17:43:12,POST,/administrator/index.php,HTTP/1.1,200.0,4498,NaN,NaN,0
2,192.241.220.166,NaN,NaN,2020-10-09 13:45:21,GET,/manager/text/list,HTTP/1.1,200.0,322,NaN,Mozilla/5.0 zgrab/0.x,1
3,172.20.3.63,NaN,NaN,2019-02-05 15:03:25,GET,/documents/Extranet/Madagascar/GRET/?upload_to=,HTTP/1.1,200.0,761,NaN,"Mozilla/5.0 [en] (X11, U; OpenVAS-VT 9.0.3)",0
4,3.121.24.234,NaN,NaN,2020-08-26 03:29:43,GET,/apache-log/access.log,HTTP/1.1,200.0,10682248,NaN,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3920072,111.230.228.235,NaN,NaN,2020-10-12 00:52:14,GET,/s/index.php,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,1
3920073,111.230.228.235,NaN,NaN,2020-10-12 00:43:39,GET,/pmd_online.php,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:31...,1
3920074,54.38.79.14,NaN,NaN,2020-10-10 20:39:53,GET,/phpmyadmin1/index.php?lang=en,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1
3920075,212.95.7.227,NaN,NaN,2017-04-17 11:05:08,GET,/images/phocagallery/almhuette/thumbs/phoca_th...,HTTP/1.1,200.0,5095,http://www.almhuette-raith.at/index.php?option...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0


In [7]:
dataset_clean = dataset.fillna(0)
dataset_clean = dataset_clean[dataset_clean[3].str.len() == 19]  # Remove invalid date strings

dataset_clean[0] = pandas.Categorical(dataset_clean[0]).codes
dataset_clean[1] = pandas.Categorical(dataset_clean[1]).codes
dataset_clean[2] = pandas.Categorical(dataset_clean[2]).codes
dataset_clean[3] = pandas.to_datetime(dataset_clean[3]).astype(int)/1_000_000_000

dataset_clean[4] = pandas.Categorical(dataset_clean[4]).codes
dataset_clean[5] = pandas.Categorical(dataset_clean[5]).codes
dataset_clean[6] = pandas.Categorical(dataset_clean[6]).codes
dataset_clean[7] = dataset_clean[7].astype(int)
dataset_clean[8] = pandas.to_numeric(dataset_clean[8], errors='coerce').notnull()
dataset_clean[9] = pandas.Categorical(dataset_clean[9]).codes
dataset_clean[10] = pandas.Categorical(dataset_clean[10]).codes
dataset_clean

,0,1,2,3,4,5,6,7,8,9,10,target
0,54720,0,0,1.518323e+09,2,11695,8,206,True,8769,5673,0
1,40323,0,0,1.505065e+09,5,7811,8,200,True,0,0,0
2,35585,0,0,1.602251e+09,2,73710,8,200,True,0,12563,1
3,19671,0,0,1.549379e+09,2,24858,8,200,True,0,12559,0
4,49544,0,0,1.598413e+09,2,11695,8,200,True,0,1821,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3920072,6027,0,0,1.602464e+09,2,83362,8,200,True,0,11059,1
3920073,6027,0,0,1.602463e+09,2,82694,8,200,True,0,9099,1
3920074,67279,0,0,1.602362e+09,2,82040,8,200,True,0,6447,1
3920075,44612,0,0,1.492427e+09,2,33966,8,200,True,8945,8725,0


In [8]:
dataset_clean.dtypes

0           int32
1            int8
2            int8
3         float64
4            int8
5           int32
6            int8
7           int64
8            bool
9           int16
10          int16
target      int64
dtype: object

In [9]:
train_df = dataset_clean.sample(frac=0.8, random_state=1)
test_df = dataset_clean.loc[~dataset_clean.index.isin(train_df.index)]
columns = [c for c in train_df.columns.tolist() if c != 'target']

In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
linear_model = LinearRegression()
linear_model.fit(train_df[columns], train_df['target'])

LinearRegression()

In [11]:
# This is a list of predictions based on the input columns from our test data set
predictions = linear_model.predict(test_df[columns])
predictions

array([-0.03048567,  0.90455195,  0.44453948, ...,  0.93596193,
        0.86563841,  0.86926964])

In [12]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
# This tells you how accurate the model is (how far off it is in its predictions "on average")
mse = mean_squared_error(predictions, test_df['target'])
mse

0.08851797071988055